# Tutorial for working with Altair AI Hub

In order to use this notebook you need to have the [rapidminer](https://github.com/rapidminer/python-rapidminer) package installed in your current Python environment. For installation instructions, consult the [README](https://github.com/rapidminer/python-rapidminer).


### Connect to Altair AI Hub

In order to connect to Altair AI Hub, provide credentials to be able to connect to the server. The input required from the user:
 - authentication_server: the url of the Keycloak authentication server with the /auth postfix
 - offline_token: after logging to the AI hub instance, with the correct permissions you should be able to reach your {AI Hub url}/get-token page, where you can find the value of the offline token
 - client_secret: on the above page you should be able to see the client secret for this token-tool client
If you want to fully automate the execution, you can provide both of these with additional parameters, besides other parameters as well. For more details, please consult the [documentation](https://github.com/rapidminer/python-rapidminer/blob/master/docs/api/Server.md).

In [ ]:
import rapidminer

connector = rapidminer.Server()

It will ask you about information and credentials to be able to authenticate. However, it is also possible to configure it by the following way: 

In [ ]:
import rapidminer
connector = rapidminer.Server("https://myserver.mycompany.com:8080", 
                              authentication_server="https:///myserver.mycompany.com:8081/auth", 
                              offline_token="qwert12345", client_secret="qwert12345")

### Running an Altair AI process


You may want to run a process that resides in a versioned project. Note that inputs and outputs are not allowed, as the process can only directly read from the project and potentially write back using an automatic commit and push. To run the latest version of a process in a project, use the following code:

In [ ]:
process = rapidminer.ProjectLocation("sample-dev", "processes/normalize_iris.rmp")
connector.run_process(path=process)

You can add the `project` name and `path` to the process to the `run_process` method too. You can also define `macros` and the `queue`, like the following way:

In [ ]:
connector.run_process(project="sample-dev", path="processes/normalize_iris.rmp", queue="DEFAULT", macros={"sample_size" : 100})

### Using Connections
Connections defined in the AI Hub repository are available using the following function:

In [ ]:
connections = connector.get_connections("sample-dev")
connections

Accessing the field values of these connections are possible through several ways, see examples below. Use these values to establish a connection to a database, cloud service, etc. using an appropriate Python package (e.g. _sqlalchemy_).

In [ ]:
import pandas
from sqlalchemy import create_engine
conn = connections["sample-postgres"]
postgres_str = ("postgresql://{username}:{password}@{host}:{port}/{dbname}"
    .format(username=conn.user,
        password=conn.password,
        host=conn.values["host"],
        port=conn.values["port"],
        dbname=conn.values["database"]
    )
)
cnx = create_engine(postgres_str).raw_connection()
pandas.read_sql_query("SELECT * FROM test_date_types_1", con=cnx)

### Get projects

You can also get the available projects in the Server the following way:


In [ ]:
connector.get_projects()

It returns a Connections object listing connections from the AI Hub project.

### Get queues

You can also get the queues in a Server:

In [ ]:
connector.get_queues()

It returns a JSON array of objects representing each queue with its properties.

### Web Api endpoints

The WebApi class allows you to easily score a deployed service. You only need to provide the Altair AI Hub URL and the particular service endpoint to create a class instance. After that, you can use the predict method to do scoring and get the result in pandas DataFrame format, or in JSON format (depending on the value of return_json flag). For instructions on how to deploy Web Api endpoint on Altair AI Hub, please refer to its documentation.

In [ ]:
data = [
    {
        "a1": 5.1,
        "a2": 3.5,
        "a3": 1.4,
        "a4": 0.2
    }
]
macros = {
    'macro1': 1,
    'macro2': 'value'
}

# Predicting on JSON expecting a DataFrame as a result
wa = rapidminer.WebApi(
    "http://mycompany.com:8090", "score-sales/score1")
prediction = wa.predict(data, macros)

# Predicting on pandas DataFrame expecting JSON as a result
df = pandas.DataFrame(data)
wa = rapidminer.WebApi(
    "http://mycompany.com:8090", "score-sales/score1")
prediction = wa.predict(df, macros, return_json=True)

where the Web Api endpoint is at `"score-sales/score1"` that can be applied to pandas DataFrame `data`, or list of JSON objects, with macros as parameters, and the resulting `prediction` is a pandas DataFrame as well (or JSON object). You can also define the Web Api group by defining the `web_api_group` parameter, by default it uses the `DEFAULT` one.

It can be configured to use three different authentication methods: Basic, OAuth, Long Living token.

##### Basic Auth

In [ ]:
wa = rapidminer.WebApi("http://mycompany.com:8090", "score-sales/score1",
                       authentication='basic', username="my_user", password="my_password")
prediction = wa.predict(data)

##### OAuth using Keycloak and offline_token

In [ ]:
wa = rapidminer.WebApi("https://myserver.mycompany.com:8090", "score-sales/score1",
                        authentication='oauth', authentication_server="https:///myserver.mycompany.com:8090/auth",
                          offline_token="qwert12345", client_secret="qwert12345")
prediction = wa.predict(data)

##### Long living token

In [ ]:
wa = rapidminer.WebApi("http//mycompany.com", "score-sales/score1", 
                       authentication='apitoken', apitoken="my_token")
prediction = wa.predict(data)